In [2]:
from euriai import EuriaiClient

In [4]:
client = EuriaiClient(
    api_key=apikey,
    model="gpt-4.1-nano" 
)

response = client.generate_completion(
    prompt="What is the capital of France",
    temperature=0.7,
    max_tokens=100
)

In [6]:
print(response['choices'][0]['message']['content'])

The capital of France is Paris.


In [9]:
class classifier:

    def __init__(self):
        self.client = EuriaiClient(
        api_key=apikey,
        model="gpt-4.1-mini")
    
    def ticket_classifier(self,issue,description):
        Prompt = f"""
You are a dental practice ticket classification expert. 
Analyze this ticket and classify its priority based on these STRICT RULES:

**Priority Framework:**
1. HIGH (Respond within 1 hour):
   - Emergency dental situations (severe pain, swelling, trauma)
   - Life-threatening medication reactions
   - Immediate post-operative complications

2. MEDIUM (Respond within 4 hours):
   - Appointment rescheduling requests
   - Appointment cancellation requests
   - Billing/payment disputes
   - Lab case follow-ups
   - Medication refill requests
   - Referral processing
   - Complaint resolution

3. LOW (Respond within 24 hours):
   - General inquiries about services
   - Marketing/sales questions
   - Non-urgent cancellations
   - Insurance verification requests
   - Routine prescription questions

**Special Considerations for Dental Cases:**
- Prioritize pain-related issues higher when swelling or infection is mentioned
- Elevate priority if patient mentions bleeding or trauma
- Payment issues become Medium priority when affecting treatment continuation

**Ticket Details:**
Issue Type: {issue}
Description: {description}

**Classification Rules:**
1. First determine if this matches any HIGH priority scenarios
2. If not, check against MEDIUM criteria
3. Default to LOW only if clearly non-urgent

**Output Format:**
Respond with EXACTLY one word in uppercase: HIGH, MEDIUM, or LOW
"""
        response = self.client.generate_completion(
            prompt = Prompt,
            temperature = 0.2,
            max_tokens = 20
        )

        return response['choices'][0]['message']['content']


In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import sys
import io
from datetime import datetime
if sys.stdout.encoding != 'UTF-8':
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

class Gsheetupdater():

    def __init__(self):
        self.scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        self.creds = ServiceAccountCredentials.from_json_keyfile_name("GCP_Creds.json", self.scope)
        self.client = gspread.authorize(self.creds)
        self.sheet = self.client.open("TicketSheet").sheet1

    def append_ticket(self,Ticket_ID, Name,Email,Contact,Office_Location,Issue_Type,Message,Priority):
        Date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.sheet.append_row([Date,Ticket_ID,Name, Email,Contact,Office_Location, Issue_Type,Message,Priority, "", "",""])

    
    def rownumber(self):
        """Returns the first row number (1-indexed) where Auto_Reply is empty"""
        try:
            # Get all records with header row
            records = self.sheet.get_all_records()
            
            # Find first row with empty Auto_Reply (2nd column is headers)
            for idx, row in enumerate(records, start=2):  # Start=2 because:
                # - Row 1: Headers (excluded by get_all_records)
                # - Row 2: First data row (index 0 in records)
                if not row.get('Auto_Reply', '').strip():
                    return idx
            return -1  # No matching rows found
        
        except Exception as e:
            print(f"Error finding row: {e}")
            return -1
    
    def update_ticket(self,row_number,col,reply):
        self.sheet.update_cell(row_number,col,reply)

In [11]:
class ReplyGenerator():
    def __init__(self):
        self.client = EuriaiClient(
            api_key=apikey,
            model="gpt-4.1-mini"
        )

    def generate_reply(self, issue, message, Name):
        """
        Generates a professional dental support reply
        
        Args:
            issue (str): Type of issue (e.g., "Inquiry", "Emergency", "Appointment/Rescheduling", "Payment", 
             "Billing", "Complaint", "Medication/RX/Request/Question", 
             "Marketing/Sales", "Refund", "Referrals", "Cancellation")
            message (str): Patient's detailed description
            patient_name (str): Optional patient name for personalization
            
        Returns:
            str: Generated response with proper formatting
        """
        prompt = f"""
**Role**: You are Capline Dental's AI Support Agent. Respond professionally yet compassionately.

**Patient Information**:
- Name: {Name if Name else "Patient"}
- Issue Type: {issue}
- Description: {message}

**Response Guidelines**:
1. Start with empathetic acknowledgment
2. Provide clear next steps
3. Include relevant dental advice if applicable
4. Set proper expectations for resolution time
5. End with contact information for urgent matters

**Special Cases**:
- For emergencies: Stress urgency and provide after-hours contact
- For billing: Explain documentation needed
- For appointments: Offer scheduling options

**Output Format**:
[Greeting] [Acknowledgment] [Action Plan] [Closing]

Example:
"Dear [Name], we understand your concern about... Our team will... For immediate assistance..."

**Generate Response**:
"""
        try:
            response = self.client.generate_completion(
                prompt=prompt,
                temperature=0.3,  # Slightly higher for varied but professional responses
                max_tokens=300,   # Increased for detailed replies
                top_p=0.9
            )
            
            # Extract and clean response
            full_reply = response['choices'][0]['message']['content'].strip()
            
            # Ensure proper formatting
            if not full_reply.startswith(("Dear", "Hello", "Hi")):
                full_reply = f"Dear {Name if Name else 'Patient'},\n\n{full_reply}"
                
            return full_reply
            
        except Exception as e:
            error_msg = f"We encountered an error generating your response. Our team has been notified."
            print(f"AI Generation Error: {str(e)}")
            return error_msg

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
if sys.stdout.encoding != 'UTF-8':
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')




class Emailreply():

    def __init__(self):
        self.email = email
        self.password = psswd

    def email_sender(self,to_email,subject,body):
        msg = MIMEMultipart()
        msg['From'] = self.email
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain')) 



        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.set_debuglevel(0)  # Set to 1 to enable full SMTP debug
        server.starttls()
        server.login(self.email, self.password)
        server.send_message(msg)
        server.quit()
        

In [ ]:
classify = classifier()
update = Gsheetupdater()
replygenerator = ReplyGenerator()
sender = Emailreply()


ticket_id = 'CAP-123456'
name = 'john doe'
email = 'abcxyz@gmail.com'
Contact = 760000000
office= 'Aransas'
issue = 'Emergency'
Message = "I have got my Capping done last friday, but suddenly there is some bleeding from my gums and pain while eating"
Priority = classify.ticket_classifier(issue,Message)
update.append_ticket(ticket_id,name,email,Contact,office,issue,Message,Priority)
row_number = update.rownumber()
reply = replygenerator.generate_reply(issue,Message,name)
update.update_ticket(row_number,10,reply)
subject = (f"{issue} issue faced at {office} office")
sender.email_sender(email,subject,reply)
print("Email send sucessfully")

C:\Users\priya\AppData\Local\Temp\ipykernel_18560\17926297.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  update = Gsheetupdater()


Email send sucessfully
